In [1]:
#Importación de librerias útiles
import pandas
#corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')

#Librerias cientificas scipy, numpy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para aplicar Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')

## Eliminar comentarios con menos de tres votos

In [18]:
#Listas para salvar los tweets filtrados
id_f = []
text_f = []
account_id_f=[]
nh_f=[]
sh1_f=[]
sh2_f=[]
sh3_f=[]
sh4_f=[]
sh5_f=[]

#Listas del corpus
vec_id=corpus['id'][:]
vec_text=corpus['text'][:]
vec_account_id=corpus['account_id'][:]
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]

for i in range(len(corpus)):
    if vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= 3:
        id_f.append(vec_id[i])
        text_f.append(vec_text[i])
        account_id_f.append(vec_account_id[i])
        nh_f.append(vec_no_humor[i])
        sh1_f.append(vec_e1[i])
        sh2_f.append(vec_e2[i])
        sh3_f.append(vec_e3[i])
        sh4_f.append(vec_e4[i])
        sh5_f.append(vec_e5[i])
        



In [35]:
print("Tweets con al menos tres votos", len(text_f))
d = {'id' : id_f,
    'text' : text_f,
    'account_id': account_id_f,
    'n':nh_f, 
    '1':sh1_f,
    '2':sh2_f,
    '3':sh3_f,
    '4':sh4_f,
    '5':sh5_f
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro1.csv')
print("Tweets guardados en corpus_filtro1.csv")

Tweets con al menos tres votos 3438
Tweets guardados en corpus_filtro1.csv


## La estructura con la que se va a trabajar ahora se levanta del archivo corpus_filtro1.cvs, que contiene solo los tweets con a los sumo tres votos

# La siguiente actividad es eliminar Hashtags

In [59]:
text_f = []

corpus_filtro1 = pandas.read_csv("corpus_filtro1.csv",encoding='utf-8')

pattern_hashtag = re.compile(r'#.+?\b')



for i in range(len(corpus_filtro1)):
    text_f.append(re.sub(pattern_hashtag, "", corpus_filtro1['text'][i]))


In [60]:
print("Tweets sin hashtags")
d = {'id' : corpus_filtro1['id'][:],
    'text' : text_f,
    'account_id': corpus_filtro1['account_id'][:],
    'n':corpus_filtro1['n'][:], 
    '1':corpus_filtro1['1'][:],
    '2':corpus_filtro1['2'][:],
    '3':corpus_filtro1['3'][:],
    '4':corpus_filtro1['4'][:],
    '5':corpus_filtro1['5'][:]
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro2.csv')
print("Tweets guardados en corpus_filtro2.csv")

Tweets sin hashtags
Tweets guardados en corpus_filtro2.csv


# La estructura con la que se va a trabajar ahora se levanta desde corpus_filtro2.csv. Esta estructura contiene los tweets que tienen al menos tres votos y se han eliminado del texto los hashtags.

# A continuación se agregará al corpus la información de si es humorístico o no y manteniendo solo los campos 1,2,3,4,5

In [2]:
corpus_filtro2 = pandas.read_csv("corpus_filtro2.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
text = []
category=[]
h1=[]
h2=[]
h3=[]
h4=[]
h5=[]

#Listas del corpus
vec_text=corpus_filtro2['text'][:]
vec_no_humor =corpus_filtro2['n'][:]
vec_e1 =corpus_filtro2['1'][:]
vec_e2 =corpus_filtro2['2'][:]
vec_e3 =corpus_filtro2['3'][:]
vec_e4 =corpus_filtro2['4'][:]
vec_e5 =corpus_filtro2['5'][:]

for i in range(len(corpus_filtro2)):
    if  vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= vec_no_humor[i]:
        category.append(1)
    else:
        category.append(0)
    text.append(vec_text[i])
    h1.append(vec_e1[i])
    h2.append(vec_e2[i])
    h3.append(vec_e3[i])
    h4.append(vec_e4[i])
    h5.append(vec_e5[i])

In [3]:
print("Guardando corpus y categorias")
d = {'text' : text,
    'category': category,
     '1':h1,
     '2':h2,
     '3':h3,
     '4':h4,
     '5':h5
    }
df = pandas.DataFrame(d, columns = ['text','category','1','2','3','4','5'])
df.to_csv('corpus_filtro3.csv')
print("Tweets guardados en corpus_filtro3.csv")

Guardando corpus y categorias
Tweets guardados en corpus_filtro3.csv


# Se agregará información de POS-tag al texto, para considerar las POS-tag como features 

In [11]:
corpus_filtro3 = pandas.read_csv("corpus_filtro3.csv",encoding='utf-8')
text_list=corpus_filtro3['text'][:]


print("Chiste 1 ",len(text_list[2903]), "\n Chiste 2", not math.isnan(text_list[2904]), "\n Chiste 3", len(text_list[2905]) )

Chiste 1  137 
 Chiste 2 False 
 Chiste 3 72


In [27]:
corpus_filtro3 = pandas.read_csv("corpus_filtro3.csv",encoding='utf-8')
text_list=corpus_filtro3['text'][:]

pattern_pos = re.compile(r'pos="(.*?)"')
num_sentences=len(corpus_filtro3)
numsentence=1
pos=0
for d in text_list:
    if(type(d) == str):
        xml = analyzer.run(d.encode(), 'flush')
        print(numsentence, " ", math.floor( numsentence/num_sentences*100),"%", end="\r")
        for sentence in xml:        
            for token in sentence:
                token_byte=etree.tostring(token)
                m = re.search(pattern_pos, token_byte.decode())
                if m is not None:
                    text_list[ pos] = text_list[pos] + " " + m.group(1)
        #print(text_list[2903 + pos])
    #else:
    #    print("Es nan")
    pos+=1
    numsentence+=1

In [29]:
print("Guardando corpus y categorias")
d = {'text' : text_list,
    'category': corpus_filtro3['category'][:],
     '1':corpus_filtro3['1'][:],
     '2':corpus_filtro3['2'][:],
     '3':corpus_filtro3['3'][:],
     '4':corpus_filtro3['4'][:],
     '5':corpus_filtro3['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', '1','2','3','4','5'])
df.to_csv('corpus_filtro4.csv')
print("Tweets guardados en corpus_filtro4.csv")

Guardando corpus y categorias
Tweets guardados en corpus_filtro4.csv


# Separación de datos de entrenamiento y datos de desarrollo

In [3]:
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')
num_dev_tweets=math.floor(len(corpus_filtro4)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4) - 1, size=num_dev_tweets)

print(num_dev_tweets)
print(random_samples)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4['text'][:]
vec_category=corpus_filtro4['category'][:]

for i in range( len(corpus_filtro4)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])
        
print("Cantidad de tweets para entrenamiento", len(text_train))
print("Cantidad de tweets para desarrollo", len(text_dev))


687
[1241 2566 1048 2353 2749 2111 1410 3291  205 3063 1622  274 1409 1370 2190
  969  768 3230 1048 1467  171 2437 2480 1423 2485  551 1616 2287 2225  546
 1292  164  186 2532 2625  867  947  451 3372 2746 2230 1531 3062 3096  709
 1498 2892  376  741 1232 2092  494 3311 1917  459  937 2836 2046 1908 1135
 1877 2306 2886 1843 2316 2843 3162 1778 2750 2472 2181 1126 2673 2528 2512
  891  954  256 1621 3075   29  578 1024  490 2799 3266 2555  447 2960 1434
 2571 1199  749 3212 1947 1719 3363 2031  601 2254 3190  964 1086 3034 1183
 2746 1036  443 3090 1949 1627 2176 2249 1696  266 3299  187  591 1241  827
  325 3425 2483 2341  869  668 2415  501 1664   57 1274 2939 2719 2836 2220
  724 2954 1192 2196   14 3283  375  544 1872 2574 1510 2111  104 1279 1269
 2949  911  912 1112 2904  757 3138 2916 2535  199 1127  256 3282 3328  705
 1685 1443  397 2518 1636   33 2268  748  978  851 1905  800 2904  531  426
 1730 2632 2952  275 1836 2636  124 1258  815  197 1695 1069 1963   54 1216
  367 28

In [4]:
print("Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo")
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset.csv')
print("Tweets guardados en corpus_filtro5_trainingset.csv")
d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset.csv')
print("Tweets guardados en corpus_filtro5_devset.csv")

Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo
Tweets guardados en corpus_filtro5_trainingset.csv
Tweets guardados en corpus_filtro5_devset.csv


# Prediccion de tweets naive bayes

In [5]:
vectorizer = CountVectorizer(stop_words='english')

In [23]:
corpus_filtro5_trainingset = pandas.read_csv("corpus_filtro5_trainingset.csv",encoding='utf-8')
text_list=corpus_filtro5_trainingset['text'][:]
category_list=corpus_filtro5_trainingset['category'][:]
pos=0
for t in text_list:
    if type(t) != str:
        text_list[pos]="NaN"
    pos+=1
train_features = vectorizer.fit_transform(text_list)

In [24]:
corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
text_devlist=corpus_filtro5_devset['text'][:]
category_devlist=corpus_filtro5_devset['category'][:]
pos=0
for t in text_devlist:
    if type(t) != str:
        text_devlist[pos]="NaN"
    pos+=1
test_features = vectorizer.transform(text_devlist)


In [25]:
nb = MultinomialNB()
nb.fit(train_features , category_list)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
predictions = nb.predict(test_features)

In [27]:
from sklearn.naive_bayes import MultinomialNB
print(predictions)
#nb.predict_proba(test_features)

[1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1
 0 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0
 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0
 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1
 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1
 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 1 1 1 1 0 0 0 0 0 0 0 1 